In [1]:
!pip install pandas numpy matplotlib pyarrow scikit-learn tkan temporal_linear_network scipy "jax[cuda12]" xlrd -U

# Run on: NVIDIA RTX-4090 + Ryzen 5900X with cuda 12.6

In [2]:
# Standard library imports
import os
import time
from urllib.request import urlretrieve
from abc import ABC, abstractmethod

import jax
print(jax.devices())

# Set Keras backend
BACKEND = 'jax'  # You can use any backend here
os.environ['KERAS_BACKEND'] = BACKEND

# Third-party imports
import numpy as np
import pandas as pd
from scipy import stats, ndimage
from scipy.ndimage import zoom
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Keras imports
import keras
from keras import ops
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.regularizers import l2
from keras.models import Sequential, Model
from keras.optimizers import Optimizer, Adam
from keras.applications import MobileNetV3Small
from keras.layers import Dense, GlobalAveragePooling2D
from keras.datasets import fashion_mnist, cifar10, cifar100, california_housing

[CudaDevice(id=0)]


In [3]:
N_MAX_EPOCHS = 1000
BATCH_SIZE = 64

def early_stopping_callback():
    return keras.callbacks.EarlyStopping(
        monitor="val_loss",
        min_delta=0.00001,
        patience=15,
        mode="min",
        restore_best_weights=True,
        start_from_epoch=0,
    )

def lr_callback():
    return keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.25,
        patience=6,
        mode="min",
        min_delta=0.00001,
        min_lr=0.000025,
        verbose=0,
    )

def get_callbacks():
    return [early_stopping_callback(), lr_callback(), keras.callbacks.TerminateOnNaN()]

In [4]:
class ScalingStrategy(ABC):
    def __init__(self, **kwargs):
        self.hyperparameters = kwargs

    @abstractmethod
    def compute_scaling_factor(self, connections, gradients, layer_info, model_info):
        raise NotImplementedError


class AdditiveMinMaxMedianConnectionScaling(ScalingStrategy):
    def compute_scaling_factor(self, connections, gradients, layer_info, model_info):
        scaling_factor = self.hyperparameters.get('scaling_factor', 0.95)
        if model_info['min_connections'] == model_info['max_connections']:
            return 1.0
        if connections <= model_info['median_connections']:
            return 1 + scaling_factor * (
                (model_info['median_connections'] - connections) / 
                (model_info['median_connections'] - model_info['min_connections'])
            )
        else:
            return 1 - scaling_factor * (
                (connections - model_info['median_connections']) / 
                (model_info['max_connections'] - model_info['median_connections'])
            )
            
    def get_config(self):
        return {"scaling_factor": self.hyperparameters.get('scaling_factor', 5.0)}

    @classmethod
    def from_config(cls, config):
        return cls(scaling_factor=config['scaling_factor'])

class MultiplicativeMinMaxMedianConnectionScaling(ScalingStrategy):
    def compute_scaling_factor(self, connections, gradients, layer_info, model_info):
        scaling_factor = self.hyperparameters.get('scaling_factor', 5.0)
        if model_info['min_connections'] == model_info['max_connections']:
            return 1.0
        
        # Normalize the connections to a range between -1 and 1
        if connections <= model_info['median_connections']:
            normalized = (model_info['median_connections'] - connections) / (model_info['median_connections'] - model_info['min_connections'])
        else:
            normalized = (connections - model_info['median_connections']) / (model_info['max_connections'] - model_info['median_connections'])
        
        # Use an exponential function to map the normalized value to a scaling factor
        # This will map -1 to 1/scaling_factor, 0 to 1, and 1 to scaling_factor
        return ops.exp(ops.log(scaling_factor) * normalized)

    def get_config(self):
        return {"scaling_factor": self.hyperparameters.get('scaling_factor', 5.0)}

    @classmethod
    def from_config(cls, config):
        return cls(scaling_factor=config['scaling_factor'])
        
class DepthConnectionScaling(ScalingStrategy):
    def compute_scaling_factor(self, connections, gradients, layer_info, model_info):
        scaling_factor = self.hyperparameters.get('scaling_factor', 1.0)
        return (1. + scaling_factor) ** ((model_info['total_depth'] - (1 + layer_info['depth'])) / model_info['total_depth'])

    def get_config(self):
        return {"scaling_factor": self.hyperparameters.get('scaling_factor', 5.0)}

    @classmethod
    def from_config(cls, config):
        return cls(scaling_factor=config['scaling_factor'])
        

class ConnectionAwareAdam(Optimizer):
    def __init__(
        self,
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7,
        amsgrad=False,
        scaling_strategy=AdditiveMinMaxMedianConnectionScaling(scaling_factor=0.9),
        name="ConnectionAwareAdam",
        **kwargs
    ):
        super().__init__(learning_rate=learning_rate, name=name, **kwargs)
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.amsgrad = amsgrad
        self.scaling_strategy = scaling_strategy
        self.connection_counts = {}
        self.min_connections = float('inf')
        self.max_connections = 0
        self.median_connections = 0
        self.model_info = {}

    def build(self, var_list):
        super().build(var_list)
        self._momentums = []
        self._velocities = []
        for var in var_list:
            self._momentums.append(
                self.add_variable_from_reference(reference_variable=var, name="m")
            )
            self._velocities.append(
                self.add_variable_from_reference(reference_variable=var, name="v")
            )
        if self.amsgrad:
            self._velocity_hats = []
            for var in var_list:
                self._velocity_hats.append(
                    self.add_variable_from_reference(reference_variable=var, name="vhat")
                )
        self._calculate_connection_counts(var_list)

    def _calculate_connection_counts(self, var_list):
        all_connections = []
        for var in var_list:
            if len(var.shape) > 1:  # Only consider variables with more than 1 dimension (i.e., not biases)
                connections = np.prod(var.shape)
                self.connection_counts[var.name] = connections
                all_connections.append(connections)

        if all_connections:
            self.min_connections = min(all_connections)
            self.max_connections = max(all_connections)
            self.median_connections = np.median(all_connections)
        else:
            self.min_connections = self.max_connections = self.median_connections = 1  # Default to avoid division by zero

        self.model_info = {
            'min_connections': self.min_connections,
            'max_connections': self.max_connections,
            'median_connections': self.median_connections,
            'total_depth': len(var_list),
        }

    def update_step(self, gradient, variable, learning_rate):
        lr = ops.cast(learning_rate, variable.dtype)
        gradient = ops.cast(gradient, variable.dtype)
        local_step = ops.cast(self.iterations + 1, variable.dtype)
        beta_1_power = ops.power(ops.cast(self.beta_1, variable.dtype), local_step)
        beta_2_power = ops.power(ops.cast(self.beta_2, variable.dtype), local_step)

        m = self._momentums[self._get_variable_index(variable)]
        v = self._velocities[self._get_variable_index(variable)]

        # Apply scaling strategy
        connections = self.connection_counts.get(variable.name, 0)
        layer_info = {
            'depth': self._get_variable_index(variable),
        }
        scaling_factor = self.scaling_strategy.compute_scaling_factor(
            connections, gradient, layer_info, self.model_info
        )

        alpha = lr * ops.sqrt(1 - beta_2_power) / (1 - beta_1_power)
        alpha *= ops.cast(scaling_factor, variable.dtype)

        m.assign(self.beta_1 * m + (1 - self.beta_1) * gradient)
        v.assign(self.beta_2 * v + (1 - self.beta_2) * ops.square(gradient))

        if self.amsgrad:
            v_hat = self._velocity_hats[self._get_variable_index(variable)]
            v_hat.assign(ops.maximum(v_hat, v))
            v = v_hat

        variable.assign_sub(alpha * m / (ops.sqrt(v) + self.epsilon))

    def get_config(self):
        config = super().get_config()
        config.update({
            "beta_1": self.beta_1,
            "beta_2": self.beta_2,
            "epsilon": self.epsilon,
            "amsgrad": self.amsgrad,
            "scaling_strategy": self.scaling_strategy.__class__.__name__,
        })
        return config

In [5]:
def resnet_block(x, filters, kernel_size=3, strides=1, conv_shortcut=False, name=None):
    shortcut = x
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=strides, name=name + '_0_conv')(shortcut)
        shortcut = layers.BatchNormalization(name=name + '_0_bn')(shortcut)
    
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), name=name + '_1_conv')(x)
    x = layers.BatchNormalization(name=name + '_1_bn')(x)
    x = layers.Activation('relu', name=name + '_1_relu')(x)
    
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), name=name + '_2_conv')(x)
    x = layers.BatchNormalization(name=name + '_2_bn')(x)
    
    x = layers.Add(name=name + '_add')([shortcut, x])
    x = layers.Activation('relu', name=name + '_out')(x)
    return x

def create_resnet_cifar(input_shape, num_classes, depth=20):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (e.g., 20, 32, 44)')
    
    num_blocks = (depth - 2) // 6
    
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, 3, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), name='conv1')(inputs)
    x = layers.BatchNormalization(name='conv1_bn')(x)
    x = layers.Activation('relu', name='conv1_relu')(x)
    
    for i in range(3):
        for j in range(num_blocks):
            strides = 1
            if i > 0 and j == 0:  # first block of stages 2 and 3
                strides = 2
            x = resnet_block(x, 16 * (2**i), strides=strides, conv_shortcut=(j==0 and i>0), name=f'block{i+1}_{j+1}')
    
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(num_classes, activation='softmax', name='classifier')(x)
    
    model = Model(inputs, x, name='resnet_cifar')
    return model

def create_mobilenet_model(input_shape, num_classes):
    base_model = MobileNetV3Small(input_shape=input_shape, include_top=False, weights=None)
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Unfreeze all layers for full retraining
    for layer in model.layers:
        layer.trainable = True
    
    return model

def create_mlp_model(input_shape, hidden_layers):
    model = Sequential()
    model.add(layers.Input(shape=input_shape))
    for units in hidden_layers:
        model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dense(1))  # For regression tasks
    return model

def create_mlp_model(input_shape, hidden_layers):
    model = keras.Sequential()
    model.add(layers.Input(shape=input_shape))
    for units in hidden_layers:
        model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dense(1))  # For regression tasks
    return model

def resize_image(image, target_size):
    zoom_factors = (target_size[0] / image.shape[0], target_size[1] / image.shape[1], 1)
    return ndimage.zoom(image, zoom_factors, order=1)


def load_dataset(name):
    if name == 'fashion_mnist':
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        # Upscale to 56x56 (2x)
        x_train = np.array([zoom(img, (2, 2)) for img in x_train])
        x_test = np.array([zoom(img, (2, 2)) for img in x_test])
        # Add channel dimension and convert to RGB for MobileNetV3
        x_train = np.stack((x_train,)*3, axis=-1) / 255.
        x_test = np.stack((x_test,)*3, axis=-1) / 255.
        # Preprocess for MobileNetV2
        x_train = keras.applications.mobilenet_v3.preprocess_input(x_train)
        x_test = keras.applications.mobilenet_v3.preprocess_input(x_test)
    elif name == 'cifar10' or name == 'cifar100':
        if name == 'cifar10':
            (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
        else:
            (x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
        
        # Normalize pixel values
        x_train = x_train.astype('float32') / 255.
        x_test = x_test.astype('float32') / 255.
        
        # Subtract per-pixel mean
        pixel_mean = np.mean(x_train, axis=0)
        x_train -= pixel_mean
        x_test -= pixel_mean
        
        return x_train, x_test, y_train, y_test
    elif name == 'california':
        (x_train, y_train), (x_test, y_test) = california_housing.load_data()
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        # Scale target (important for regression tasks)
        scaler_y = StandardScaler()
        y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
        y_test = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
        
    else:
        raise ValueError(f"Unknown dataset: {name}")
    
    return x_train, x_test, y_train, y_test

def train_and_evaluate(model, X_train, y_train, x_test, y_test, optimizer, run, is_classification=False):
    if is_classification:
        loss = 'sparse_categorical_crossentropy'
        metrics = ['accuracy']
    else:
        loss = 'mse'
        metrics = ['mae']

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    start_time = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=N_MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        callbacks=get_callbacks(),
        verbose=0
    )
    end_time = time.time()
    
    train_time = end_time - start_time
    epochs_to_converge = len(history.history['loss'])

    if is_classification:
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
        return {
            'run': run,
            'test_accuracy': test_acc,
            'test_loss': test_loss,
            'train_time': train_time,
            'epochs_to_converge': epochs_to_converge
        }
    else:
        test_mse, test_mae = model.evaluate(x_test, y_test, verbose=0)
        return {
            'run': run,
            'test_rmse': np.sqrt(test_mse),
            'test_mae': test_mae,
            'train_time': train_time,
            'epochs_to_converge': epochs_to_converge
        }

def run_benchmark(results = [], start_learning_rate=0.001):
    datasets = ['cifar10', 'cifar100']
    mlp_hidden_layers_configs = [[64, 32], [128, 64, 32], [128, 128, 64], [256, 128, 64, 32], [1024, 256, 64, 16]]
    n_runs = 30

    keras_optimizers = [
        ('Adam', lambda: optimizers.Adam(learning_rate=start_learning_rate)),
        ('AdamW', lambda: optimizers.AdamW(learning_rate=start_learning_rate)),
        ('Adamax', lambda: optimizers.Adamax(learning_rate=start_learning_rate)),
        ('Nadam', lambda: optimizers.Nadam(learning_rate=start_learning_rate)),
    ]

    connection_aware_strategies = [
        'AdditiveMinMaxMedianConnectionScaling', 'MultiplicativeMinMaxMedianConnectionScaling', 'DepthConnectionScaling',
    ]

    for dataset_name in datasets:
        print(f'On dataset: {dataset_name}')
        X_train, X_test, y_train, y_test = load_dataset(dataset_name)
        
        if dataset_name == 'fashion_mnist':
            model_configs = [('MobileNetV3Small', create_mobilenet_model, (56, 56, 3), 10)]
            is_classification = True
        elif dataset_name == 'cifar10' or dataset_name == 'cifar100':
            num_classes = 10 if dataset_name == 'cifar10' else 100
            model_configs = [('ResNet20', create_resnet_cifar, (32, 32, 3), num_classes)]
            is_classification = True
        else:  # california
            model_configs = [('MLP', create_mlp_model, X_train.shape[1:], hidden_layers) for hidden_layers in mlp_hidden_layers_configs]
            is_classification = False

        for model_name, model_func, *model_args in model_configs:
            print(f'Using model: {model_name}')
            
            initial_model = model_func(*model_args)
            initial_weights = initial_model.get_weights()
            
            for run in range(n_runs):
                print(f'Run: {run + 1}/{n_runs}')
                
                # Keras optimizers
                for opt_name, opt_func in keras_optimizers:
                    model = keras.models.clone_model(initial_model)
                    model.set_weights(initial_weights)
                    optimizer = opt_func()
                    result = train_and_evaluate(model, X_train, y_train, X_test, y_test, optimizer, run, is_classification)
                    
                    result.update({
                        'dataset': dataset_name,
                        'model': model_name,
                        'optimizer': opt_name,
                        'model_args': model_args,
                        'scaling_strategy': 'N/A',
                        'scaling_factor': 'N/A'
                    })
                    print(result)
                    results.append(result)
                    del model

                # ConnectionAwareAdam
                for strategy_name in connection_aware_strategies:
                    model = keras.models.clone_model(initial_model)
                    model.set_weights(initial_weights)
                    
                    if strategy_name == 'AdditiveMinMaxMedianConnectionScaling':
                        strategy = AdditiveMinMaxMedianConnectionScaling()
                    elif strategy_name == 'MultiplicativeMinMaxMedianConnectionScaling':
                        strategy = MultiplicativeMinMaxMedianConnectionScaling()
                    elif strategy_name == 'DepthConnectionScaling':
                        strategy = DepthConnectionScaling()
                    
                    optimizer = ConnectionAwareAdam(learning_rate=start_learning_rate, scaling_strategy=strategy)
                    result = train_and_evaluate(model, X_train, y_train, X_test, y_test, optimizer, run, is_classification)
                    result.update({
                        'dataset': dataset_name,
                        'model': model_name,
                        'optimizer': 'ConnectionAwareAdam',
                        'scaling_strategy': strategy_name,
                        'model_args': model_args,
                    })
                    print(result)
                    results.append(result)
                    del model

    df_results = pd.DataFrame(results)
    return df_results




def perform_statistical_tests(df):
    df['model_args'] = df['model_args'].astype(str)
    grouped = df.groupby(['dataset', 'model_args', 'model', 'optimizer', 'scaling_strategy',])
        
    def get_metric_and_direction(dataset):
        if dataset in ['fashion_mnist', 'cifar100', 'cifar10']:
            return 'test_accuracy', 1  # Higher is better
        elif dataset == 'california':
            return 'test_rmse', -1  # Lower is better
        else:
            raise ValueError(f"Unknown dataset: {dataset}")
    
    summary_list = []
    statistical_results = []
    
    for dataset in df['dataset'].unique():
        metric, improvement_direction = get_metric_and_direction(dataset)
        
        dataset_summary = grouped.agg({
            metric: ['mean', 'std'],
            'train_time': ['mean', 'std'],
            'epochs_to_converge': ['mean', 'std']
        }).reset_index()
        dataset_summary = dataset_summary[dataset_summary['dataset']==dataset]
        
        dataset_summary.columns = ['dataset', 'model_args', 'model', 'optimizer', 'scaling_strategy', 
                                   f'{metric}_mean', f'{metric}_std', 'time_mean', 'time_std', 
                                   'epochs_mean', 'epochs_std']
        summary_list.append(dataset_summary)
        
        for model in df['model'].unique():
            for model_args in df['model_args'].unique():
                adam_results = df[(df['dataset'] == dataset) & 
                                  (df['model'] == model) & 
                                  (df['optimizer'] == 'Adam') &
                                  (df['model_args'] == model_args)]
                
                if adam_results.empty:
                    continue
                
                adam_metric_mean = adam_results[metric].mean()
                adam_time_mean = adam_results['train_time'].mean()
                
                for optimizer in df['optimizer'].unique():
                    if optimizer == 'Adam':
                        continue
                    
                    if optimizer == 'ConnectionAwareAdam':
                        for strategy in df[df['optimizer'] == 'ConnectionAwareAdam']['scaling_strategy'].unique():
                            opt_results = df[(df['dataset'] == dataset) & 
                                             (df['model'] == model) & 
                                             (df['optimizer'] == 'ConnectionAwareAdam') & 
                                             (df['scaling_strategy'] == strategy) &
                                             (df['model_args'] == model_args)]
                            
                            if opt_results.empty:
                                continue
                            
                            opt_metric_mean = opt_results[metric].mean()
                            opt_time_mean = opt_results['train_time'].mean()
                            
                            metric_improvement = ((opt_metric_mean - adam_metric_mean) / adam_metric_mean) * 100 * improvement_direction
                            time_improvement = ((opt_time_mean - adam_time_mean) / adam_time_mean) * 100 * -1  # Negative because lower time is better
                            
                            metric_t_stat, metric_p_value = stats.ttest_ind(adam_results[metric], opt_results[metric]) 
                            time_t_stat, time_p_value = stats.ttest_ind(adam_results['train_time'], opt_results['train_time'])

                            if improvement_direction > 0:
                                metric_t_stat *= -1
                            
                            statistical_results.append({
                                'dataset': dataset,
                                'model': model,
                                'optimizer': 'ConnectionAwareAdam',
                                'scaling_strategy': strategy,
                                'model_args': model_args,
                                'metric': metric,
                                f'{metric}_improvement_%': metric_improvement,
                                f'{metric}_t_statistic': metric_t_stat,
                                f'{metric}_p_value': metric_p_value,
                                'time_improvement_%': time_improvement,
                                'time_t_statistic': time_t_stat,
                                'time_p_value': time_p_value
                            })
                    else:
                        opt_results = df[(df['dataset'] == dataset) & 
                                         (df['model'] == model) & 
                                         (df['optimizer'] == optimizer) &
                                         (df['model_args'] == model_args)]
                        
                        if opt_results.empty:
                            continue
                        
                        opt_metric_mean = opt_results[metric].mean()
                        opt_time_mean = opt_results['train_time'].mean()
                        
                        metric_improvement = ((opt_metric_mean - adam_metric_mean) / adam_metric_mean) * 100 * improvement_direction
                        time_improvement = ((opt_time_mean - adam_time_mean) / adam_time_mean) * 100 * -1  # Negative because lower time is better
                        
                        metric_t_stat, metric_p_value = stats.ttest_ind(adam_results[metric], opt_results[metric])
                        time_t_stat, time_p_value = stats.ttest_ind(adam_results['train_time'], opt_results['train_time'])

                        if improvement_direction > 0:
                            metric_t_stat *= -1
                        
                        statistical_results.append({
                            'dataset': dataset,
                            'model': model,
                            'optimizer': optimizer,
                            'scaling_strategy': 'N/A',
                            'model_args': model_args,
                            'metric': metric,
                            f'{metric}_improvement_%': metric_improvement,
                            f'{metric}_t_statistic': metric_t_stat,
                            f'{metric}_p_value': metric_p_value,
                            'time_improvement_%': time_improvement,
                            'time_t_statistic': time_t_stat,
                            'time_p_value': time_p_value
                        })
    
    summary = pd.concat(summary_list, axis=0).sort_values(['dataset', 'model_args', 'optimizer', 'scaling_strategy'])
    statistical_results = pd.DataFrame(statistical_results).sort_values(['dataset', 'model_args', 'optimizer', 'scaling_strategy'])
    return summary, statistical_results

In [6]:
# Run the benchmark
results = []
results = run_benchmark(results=results)

# Save raw results to CSV
results.to_csv('benchmark_results.csv', index=False)

On dataset: cifar10
Using model: ResNet20
Run: 1/30
{'run': 0, 'test_accuracy': 0.8137000203132629, 'test_loss': 0.8269512057304382, 'train_time': 63.937613010406494, 'epochs_to_converge': 28, 'dataset': 'cifar10', 'model': 'ResNet20', 'optimizer': 'Adam', 'model_args': [(32, 32, 3), 10], 'scaling_strategy': 'N/A', 'scaling_factor': 'N/A'}
{'run': 0, 'test_accuracy': 0.7891000509262085, 'test_loss': 0.8959359526634216, 'train_time': 65.0179033279419, 'epochs_to_converge': 29, 'dataset': 'cifar10', 'model': 'ResNet20', 'optimizer': 'AdamW', 'model_args': [(32, 32, 3), 10], 'scaling_strategy': 'N/A', 'scaling_factor': 'N/A'}
{'run': 0, 'test_accuracy': 0.7130000591278076, 'test_loss': 1.0761616230010986, 'train_time': 60.98489689826965, 'epochs_to_converge': 27, 'dataset': 'cifar10', 'model': 'ResNet20', 'optimizer': 'Adamax', 'model_args': [(32, 32, 3), 10], 'scaling_strategy': 'N/A', 'scaling_factor': 'N/A'}
{'run': 0, 'test_accuracy': 0.7993000149726868, 'test_loss': 0.954172492027282

In [9]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Perform statistical tests
summary, statistical_results = perform_statistical_tests(results)

# Save processed results to CSV
summary.to_csv('benchmark_summary.csv', index=False)
statistical_results.to_csv('benchmark_statistical_results.csv', index=False)

# Display results
print("Summary:")
display(summary)
print("\nStatistical Results:")
display(statistical_results)

Summary:


,dataset,model_args,model,optimizer,scaling_strategy,test_accuracy_mean,test_accuracy_std,time_mean,time_std,epochs_mean,epochs_std
0,cifar10,"[(32, 32, 3), 10]",ResNet20,Adam,N/A,0.798137,0.008011,63.741011,3.621199,28.633333,1.938420
1,cifar10,"[(32, 32, 3), 10]",ResNet20,AdamW,N/A,0.790730,0.014213,64.706269,7.060076,28.700000,3.843041
2,cifar10,"[(32, 32, 3), 10]",ResNet20,Adamax,N/A,0.689013,0.014790,54.893721,4.432297,23.733333,2.448551
3,cifar10,"[(32, 32, 3), 10]",ResNet20,ConnectionAwareAdam,AdditiveMinMaxMedianConnectionScaling,0.820840,0.005632,66.616815,4.180847,30.000000,2.259081
4,cifar10,"[(32, 32, 3), 10]",ResNet20,ConnectionAwareAdam,DepthConnectionScaling,0.815730,0.009909,66.828715,5.768914,30.066667,3.106537
5,cifar10,"[(32, 32, 3), 10]",ResNet20,ConnectionAwareAdam,MultiplicativeMinMaxMedianConnectionScaling,0.830807,0.005059,73.731047,7.375393,33.900000,4.138549
6,cifar10,"[(32, 32, 3), 10]",ResNet20,Nadam,N/A,0.791620,0.013996,67.789200,5.886707,29.000000,3.129393
7,cifar100,"[(32, 32, 3), 100]",ResNet20,Adam,N/A,0.478010,0.006201,71.828074,3.979034,32.500000,2.161577
8,cifar100,"[(32, 32, 3), 100]",ResNet20,AdamW,N/A,0.482907,0.005677,72.075177,3.584521,32.166667,1.931291
9,cifar100,"[(32, 32, 3), 100]",ResNet20,Adamax,N/A,0.395767,0.009427,79.360166,3.366233,36.566667,1.755451



Statistical Results:


,dataset,model,optimizer,scaling_strategy,model_args,metric,test_accuracy_improvement_%,test_accuracy_t_statistic,test_accuracy_p_value,time_improvement_%,time_t_statistic,time_p_value
0,cifar10,ResNet20,AdamW,N/A,"[(32, 32, 3), 10]",test_accuracy,-0.927995,-2.486435,1.580397e-02,-1.514343,-0.666314,5.078519e-01
1,cifar10,ResNet20,Adamax,N/A,"[(32, 32, 3), 10]",test_accuracy,-13.672261,-35.533442,4.619298e-41,13.880060,8.466618,1.016604e-11
3,cifar10,ResNet20,ConnectionAwareAdam,AdditiveMinMaxMedianConnectionScaling,"[(32, 32, 3), 10]",test_accuracy,2.844542,12.698118,2.163320e-18,-4.511701,-2.847816,6.079874e-03
5,cifar10,ResNet20,ConnectionAwareAdam,DepthConnectionScaling,"[(32, 32, 3), 10]",test_accuracy,2.204301,7.562433,3.324125e-10,-4.844140,-2.482949,1.594405e-02
4,cifar10,ResNet20,ConnectionAwareAdam,MultiplicativeMinMaxMedianConnectionScaling,"[(32, 32, 3), 10]",test_accuracy,4.093284,18.885685,1.887846e-26,-15.672854,-6.659555,1.091045e-08
2,cifar10,ResNet20,Nadam,N/A,"[(32, 32, 3), 10]",test_accuracy,-0.816486,-2.213314,3.082385e-02,-6.350995,-3.208190,2.176401e-03
6,cifar100,ResNet20,AdamW,N/A,"[(32, 32, 3), 100]",test_accuracy,1.024385,3.190282,2.294178e-03,-0.344020,-0.252719,8.013788e-01
7,cifar100,ResNet20,Adamax,N/A,"[(32, 32, 3), 100]",test_accuracy,-17.205359,-39.922431,6.990277e-44,-10.486278,-7.915483,8.487725e-11
9,cifar100,ResNet20,ConnectionAwareAdam,AdditiveMinMaxMedianConnectionScaling,"[(32, 32, 3), 100]",test_accuracy,2.721003,6.039085,1.174876e-07,1.665733,1.227707,2.245163e-01
11,cifar100,ResNet20,ConnectionAwareAdam,DepthConnectionScaling,"[(32, 32, 3), 100]",test_accuracy,5.480359,17.318000,1.365264e-24,-1.797759,-1.202438,2.340801e-01
